### 5.2 Whole Slide Scoring - Prediction Confidence Segmentation

Prediction confidence heatmaps were segmented to generate the CNN model-based whole-slide scores. 

First, a CNN confidence threshold was applied to the heatmaps, with only prediction confidences higher than the threshold retained, indicating the existences of plaques predicted with high confidence. Morphological opening and closing operations were then performed to smooth the masks, and prediction areas exceeding a size threshold set to eliminate CNN-noise provided the predicted quantities of Aβ pathologies (ie. cored plaque, diffuse plaque, and CAA). 

These two hyperparameters, CNN confidence threshold (core: 0.1; diffuse: 0.95; CAA: 0.9) and size threshold (cored: 100; diffuse: 1; CAA: 200), were optimized exclusively by statistical analysis on the combined training and validation sets (32 WSIs). These were then used on the hold-out test set with 30 unseen WSIs to show the generalizability. 

In [1]:
import csv
import glob, os

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import measure
from scipy import stats

from tqdm import tqdm

In [2]:
CSV_DIR = 'data/outputs/CNNscore/WSI_CERAD_AREA.csv'
HEATMAP_DIR = 'data/outputs/heatmaps'
SAVE_DIR = 'data/outputs/CNNscore/'

In [3]:
if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)

In [ ]:
file = pd.read_csv(CSV_DIR)
filenames = list(file['WSI_ID'])
img_class = ['cored', 'diffuse', 'caa']

# two hyperparameters
confidence_thresholds = [0.1, 0.95, 0.9]
pixel_thresholds = [100, 1, 200]

In [3]:
def count_blobs(mask,
               threshold=1500):
    labels = measure.label(mask, neighbors=8, background=0)
    new_mask = np.zeros(mask.shape, dtype='uint8')
    sizes = []
    
    # loop over the unique components
    for label in np.unique(labels):
        # if this is the background label, ignore it
        if label == 0:
            continue
        # otherwise, construct the label mask and count the
        # number of pixels 
        labelMask = np.zeros(mask.shape, dtype="uint8")
        labelMask[labels == label] = 255
        numPixels = cv2.countNonZero(labelMask)
        
        # if the number of pixels in the component is sufficiently
        # large, then add it to our mask of "large blobs"
        if numPixels > threshold:
            sizes.append(numPixels)
            new_mask = cv2.add(new_mask, labelMask)

    return sizes, new_mask

In [6]:
new_file = file
for index in [0,1,2]:
    preds = np.zeros(len(file))
    confidence_threshold = confidence_thresholds[index]
    pixel_threshold = pixel_thresholds[index]

    for i, WSIname in tqdm(enumerate(filenames)):
        try:
            heatmap_path = HEATMAP_DIR+'new_WSI_heatmap_{}.npy'.format(WSIname)
            h = np.load(heatmap_path)

        except:
            heatmap_path = HEATMAP_DIR+'{}.npy'.format(WSIname)
            h = np.load(heatmap_path)

        mask = h[index] > confidence_threshold
        mask = mask.astype(np.float32)

        kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(5,5))

        # Apply morphological closing, then opening operations 
        opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

        a, m = count_blobs(closing, threshold=pixel_threshold)

        preds[i] = len(a)

    print(confidence_threshold, pixel_threshold)

    new_file['CNN_{}_count'.format(img_class[index])] = preds

new_file.to_csv(SAVE_DIR+'CNN_vs_CERAD.csv', index=False)

62it [07:13,  6.99s/it]
0it [00:00, ?it/s]

0.1 100


62it [20:42, 20.05s/it]
0it [00:00, ?it/s]

0.95 1


62it [01:14,  1.21s/it]

0.9 200


In [7]:
new_file

,WSI_ID,Group,CERAD,Cored_MTG,Diffuse_MTG,CAA_MTG,Area,CNN_cored_count,CNN_diffuse_count,CNN_caa_count
0,NA4009-02_AB,train,none,1.0,2.0,0.0,5863952,96.0,4296.0,38.0
1,NA_4933_02_AB17-24,validation,none,0.0,1.0,0.0,3786178,27.0,29.0,1.0
2,NA4137-02_AB,train,sparse,0.0,1.0,0.0,5976867,15.0,22.0,2.0
3,NA_4865_02_AB1-40,train,sparse,0.0,0.0,0.0,5093794,0.0,0.0,0.0
4,NA_4871_02_AB,train,sparse,0.0,0.0,0.0,3997928,0.0,0.0,2.0
5,NA5001_2AB,train,sparse,1.0,2.0,3.0,2730917,7.0,410.0,595.0
6,NA5003_2AB,train,sparse,1.0,1.0,2.0,4108923,55.0,109.0,117.0
7,NA4144-02_AB,train,sparse,0.0,3.0,0.0,6054061,50.0,4390.0,4.0
8,NA4072-02_AB,train,moderate,1.0,2.0,0.0,8362154,40.0,9.0,0.0
9,NA4185-02_AB,train,moderate,2.0,2.0,0.0,6191811,861.0,1113.0,4.0
